In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import qiskit.quantum_info as qi

In [2]:
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')

In [3]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)

In [4]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args:
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [5]:
def U(A, phi):
    return expm(-1j*A*phi).astype('complex')

def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    for i in range(n_of_qubits):
        if i == control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i == target_q:
            list_of_ops_left.append(I())
            list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [6]:
#state preparation
state = multi_kron(I(), H(), H(), H(), X())
np.round(state, 3)
#eigenvalues to phases
etp_one = control(control_q=3,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/2))
etp_two = control(control_q=2,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/4))
etp_three = control(control_q=1,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, np.pi/8))
# print(np.round(multi_dot(state, etp_one, etp_two, etp_three), 3))
#Fourier transform
ft_one = multi_kron(I(), H(), I(), I(), I())
ft_two = control(control_q=1,
                 target_q=3,
                 n_of_qubits=5,
                 operator=Rz(-np.pi/4))
ft_three = control(control_q=1,
                   target_q=2,
                   n_of_qubits=5,
                   operator=Rz(-np.pi/2))
ft_four = multi_kron(I(), I(), H(), I(), I())
ft_five = control(control_q=2,
                  target_q=3,
                  n_of_qubits=5,
                  operator=Rz(-np.pi/2))
ft_six = multi_kron(I(), I(), I(), H(), I())
# print(np.round(multi_dot(state,
#                          etp_one, etp_two, etp_three,
#                          ft_one, ft_two, ft_three, ft_four, ft_five, ft_six), 3))
# #Ancilla qubit
aq_one = control(control_q=1,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/2)) 
aq_two = control(control_q=2,
                 target_q=0,
                 n_of_qubits=5,
                 operator=Ry(np.pi/4))
aq_three = control(control_q=3,
                   target_q=0,
                   n_of_qubits=5,
                   operator=Ry(np.pi/8)) 
# print(np.round(multi_dot(
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
# #inverse_fourier
if_one = multi_kron(I(), I(), I(), H(), I())
if_two = control(
    control_q=2,
    target_q=3,
    n_of_qubits=5,
    operator=Rz(np.pi/2))
if_three = multi_kron(I(), I(), H(), I(), I())
if_four = control(control_q=1,
                  target_q=3,
                  n_of_qubits=5,
                  operator=Rz(np.pi/4))
if_five = control(control_q=1,
                  target_q=2,
                  n_of_qubits=5,
                  operator=Rz(np.pi/2))
if_six = multi_kron(I(), I(), I(), H(), I())
if_seven = multi_kron(I(), H(), I(), I(), I())
# print(np.round(multi_dot(
#     if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
#     aq_three, aq_two, aq_one,
#     ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
#     etp_three, etp_two, etp_one,
#     state), 3))
# #phases to eigenvalues
pte_one = control(control_q=1,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/8))
pte_two = control(control_q=2,
                  target_q=4,
                  n_of_qubits=5,
                  operator=U(A, -np.pi/4))
pte_three = control(control_q=3,
                    target_q=4,
                    n_of_qubits=5,
                    operator=U(A, -np.pi/2))
final_u = multi_dot(
    pte_three, pte_two, pte_one,
    if_seven, if_six, if_five, if_four, if_three, if_two, if_one,
    aq_three, aq_two, aq_one,
    ft_six, ft_five, ft_four, ft_three, ft_two, ft_one,
    etp_three, etp_two, etp_one,
    state)
# print(np.round(final_u, 3))

In [7]:
prime_state = multi_kron(state_zero, state_zero, state_zero, state_zero, state_zero)
final_state = np.dot(final_u, prime_state)
rho = np.dot(final_state, np.conj(final_state).T)
print(np.round(rho, 5), '\n')

[[ 0.03518-0.j      -0.03043-0.00078j -0.00062-0.02543j ...
  -0.00823-0.01336j  0.01384-0.01616j  0.03618-0.00335j]
 [-0.03043+0.00078j  0.02634+0.j       0.0011 +0.02198j ...
   0.00741+0.01137j -0.01161+0.01429j -0.03122+0.0037j ]
 [-0.00062+0.02543j  0.0011 -0.02198j  0.01839+0.j      ...
   0.0098 -0.00571j  0.01144+0.01029j  0.00179+0.02621j]
 ...
 [-0.00823+0.01336j  0.00741-0.01137j  0.0098 +0.00571j ...
   0.00699-0.j       0.0029 +0.00903j -0.00719+0.01452j]
 [ 0.01384+0.01616j -0.01161-0.01429j  0.01144-0.01029j ...
   0.0029 -0.00903j  0.01287+0.j       0.01577+0.0153j ]
 [ 0.03618+0.00335j -0.03122-0.0037j   0.00179-0.02621j ...
  -0.00719-0.01452j  0.01577-0.0153j   0.03753+0.j     ]] 



In [8]:
proj = multi_kron(P0, I(), I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P0, I(), I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), I(), P1)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))
proj = multi_kron(P1, I(), I(), I(), P0)
prob = np.trace(np.dot(proj, rho))
print(np.round(prob, 3))

(0.615+0j)
(0.218+0j)
(0.126-0j)
(0.041+0j)


In [9]:
prob0 = np.trace(np.dot(multi_kron(I(), I(), I(), I(), P1), rho))
prob0

(0.740381092169176+1.0923814357674336e-18j)